# Yaml info

In [ ]:
import os
from glob import glob

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import hvplot.pandas  # noqa

#
import pynsitu as pin
from pynsitu.maps import crs

In [ ]:
## data directory 
raw_dir = '/home/datawork-lops-osi/aponte/cswot/drifters/raw'
root_dir = '/home1/datahome/mdemol/PhD/insitu_drifters_trajectories'

# drifters
campaign="drifters_CSWOT_BIOSWOT_UWA"
yaml = f"{campaign}.yaml"

cp = pin.Campaign(os.path.join(root_dir,yaml))

# plot all deployments
for label, deployment, platform, sensor, meta in cp.get_all_deployments():
    print(label, platform, sensor, deployment)

_______
# CHOOSE DATASET

In [ ]:
key = 'carthe_uwa'
df = pd.read_csv(glob(os.path.join(raw_dir, 'L0_' + key+'*'))[0], parse_dates = ['time'], dtype={"id":'string'}).set_index('id')
ids = sorted(list(df.index.unique()))

In [ ]:
df

## for svp : filter out big outliers

In [ ]:
testlat = (df.lat>37) & (df.lat<45)
testlon = (df.lon>2) & (df.lon<10)
test = testlon & testlat
df = df.where(test).dropna(subset = 'lon')

In [ ]:
# extract deployment order
#df.groupby(df.index).apply(lambda df: df.sort_values("time").iloc[0]).sort_values("time")

Test if ids in data file and yaml file are identical

In [ ]:
ids_map = {p: cp[p]["serial_number"] for p in cp if  key in p}
ids_yaml = sorted(set([id for _, id in ids_map.items()]))
ids_imap = {v: k for k, v in ids_map.items()}

print("drifter id's in data file:")
print(ids)
print("drifter id's in yaml file:")

print(ids_yaml)

flag = set(ids).issuperset(ids_yaml)
if flag:
    print("Data file and campaign yaml file agree upon drifter ids")
else:
    assert False, "Data file and campaign yaml file do not agree upon drifter ids"

---
## manually adjust deployment times first

This step is useful to infer deployment times a posteriori for each drifter individually.

In [ ]:
p, d = key+'_09', "d0"

_id = ids_map[p]
print(f"Drifter id: {_id}")

_df = df.loc[_id].set_index("time").sort_index()
_df.geo.compute_velocities(inplace=True)
_df.geo.plot_bokeh(deployments=cp[p][d], velocity=True, acceleration=True)

In [ ]:
phv, coords = _df.geo.plot_on_map(s=10, c="velocity", clim=(0,1), cmap="magma")
phv

---
## Start end positions


In [ ]:
# plot all deployments
P = []
for label, deployment, platform, sensor, meta in cp.get_all_deployments():
    P.append(platform)
P = P[1:]

In [ ]:
Pcnr = [p for p in P if 'cnr' in p]
Plops = [p for p in P if 'lops' in p]
Pcode = [p for p in P if 'code' in p]
Pscripps = [p for p in P if 'scripps' in p]
Pbcg = [p for p in P if 'bcg' in p]
Psvpogs = [p for p in P if 'svp_ogs' in p]
Psvpshom = [p for p in P if 'svp_shom' in p]
Pspotter = [p for p in P if 'spotter_lops' in p]
Puwa = [p for p in P if 'carthe_uwa' in p]
for plabel in Puwa:
    p, d = plabel, "d0"
    print(p)
    start =cp[p][d]['start'].time
    end = cp[p][d]['end'].time
    print(start,end)
    _id = cp[p]['serial_number']
    _df = df.loc[_id]
    test = (_df.time>start) & (_df.time<end)
    _df=_df.where(test).dropna(subset='lon').sort_values('time')
    print(_df.iloc[0].time,_df.iloc[-1].time, _df.iloc[0].lon, _df.iloc[0].lat, _df.iloc[-1].lon, _df.iloc[-1].lat)